<a href="https://colab.research.google.com/github/sanalsajann/crop-disease-detection/blob/main/model1custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define paths
data_dir = "/content/drive/My Drive/datasetsmall"  # Path to your dataset
save_dir = "/content/drive/My Drive/models"  # Folder to save models

# Create the folder if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

print(f"Dataset path: {data_dir}")
print(f"Models will be saved at: {save_dir}")


Dataset path: /content/drive/My Drive/datasetsmall
Models will be saved at: /content/drive/My Drive/models


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load dataset with 80-20 split
train_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Get class names
class_names = train_dataset.class_names
num_classes = len(class_names)

print(f"Classes found: {class_names}")
print(f"Number of classes: {num_classes}")


Found 35627 files belonging to 2 classes.
Using 28502 files for training.
Found 35627 files belonging to 2 classes.
Using 7125 files for validation.
Classes found: ['maize', 'tomato']
Number of classes: 2


In [ ]:
# Preprocessing to normalize images to [0, 1]
preprocessing_layer = tf.keras.layers.Rescaling(1.0 / 255)

# Apply preprocessing to train and validation datasets
train_dataset = train_dataset.map(lambda x, y: (preprocessing_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (preprocessing_layer(x), y))

print("✅ Preprocessing applied. Images normalized to [0, 1].")


✅ Preprocessing applied. Images normalized to [0, 1].


In [ ]:
# Corrected Custom CNN Model
def build_custom_cnn_v2(num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3))

    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build the corrected model
custom_cnn_model_v2 = build_custom_cnn_v2(num_classes)

print("✅ Custom CNN model built successfully!")


In [ ]:
# Define path to save models
model_save_path = "/content/drive/My Drive/models"


In [ ]:
# Train the corrected Custom CNN model
custom_cnn_history = custom_cnn_model_v2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(model_save_path + "/CustomCNN.h5", save_best_only=True),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    ]
)

print("✅ Custom CNN model training completed!")


In [ ]:
# Load the best saved version of Custom CNN
from tensorflow.keras.models import load_model

# Path to saved model
custom_cnn_path = model_save_path + "/CustomCNN.h5"

# Load the best model
custom_cnn_model = load_model(custom_cnn_path)

# Evaluate on validation set
custom_cnn_eval = custom_cnn_model.evaluate(val_dataset)
print(f"✅ Custom CNN - Final Val Accuracy: {custom_cnn_eval[1]:.4f}, Final Val Loss: {custom_cnn_eval[0]:.4f}")


In [ ]:
test_loss, test_acc = custom_cnn_model.evaluate(val_dataset)
print(f"✅ Validation Accuracy: {test_acc * 100:.2f}%")
print(f"✅ Validation Loss: {test_loss:.4f}")


223/223 ━━━━━━━━━━━━━━━━━━━━ 918s 4s/step - accuracy: 0.9934 - loss: 0.0222
✅ Validation Accuracy: 99.24%
✅ Validation Loss: 0.0250


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 35627 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices)

In [ ]:
from tensorflow.keras.models import load_model

# Path to saved model
custom_cnn_path = model_save_path + "/CustomCNN.h5"

# Load the best model
custom_cnn_model = load_model(custom_cnn_path)

In [ ]:
test_generator = test_datagen.flow_from_directory(
   data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # One-hot encoding (for categorical_crossentropy)
    shuffle=False
)


Found 35627 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model without compiling
model = load_model("path_to_your_trained_model.h5", compile=False)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
model.evaluate(test_generator, steps=1)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 854ms/step - accuracy: 0.9375 - loss: 0.1053


[0.10526745021343231, 0.9375]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Get true labels and predictions
true_labels = test_generator.classes
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys())
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Crop Disease Detection")
plt.show()

# Print classification report
print(classification_report(true_labels, predicted_labels, target_names=test_generator.class_indices.keys()))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Assuming you have a trained model
from tensorflow.keras.models import load_model


In [ ]:
# Get class names from test generator
class_names = list(test_generator.class_indices.keys())

# Print precision, recall, and F1-score
print(classification_report(true_labels, predicted_labels, target_names=class_names))
